# 验证 Libero 环境中的空间对齐 (Spatial Alignment Verification)

本 Notebook 旨在验证 Libero 环境中 3D 空间坐标（如机器人末端执行器位置）与 2D 相机图像之间的对齐准确性。

我们将通过以下步骤进行验证：
1.  初始化 Libero 仿真环境。
2.  获取当前帧的相机图像和机器人的 3D 状态（末端执行器位置）。
3.  利用相机内参和外参矩阵，将 3D 点投影到 2D 图像平面。
4.  在图像上可视化投影点。如果投影点准确地落在图像中机器人的末端位置，则说明空间信息是准确的；反之则证明存在偏差。

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from libero.libero import benchmark
from libero.libero.envs import OffScreenRenderEnv
import robosuite.utils.camera_utils as camera_utils
import robosuite.utils.transform_utils as T

# 设置中文字体以支持绘图显示中文（如果在支持的环境中）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

## 1. 初始化 Libero 环境

我们使用 `libero_spatial` 任务套件中的一个任务作为示例。

In [ ]:
# 选择任务套件和任务 ID
TASK_SUITE_NAME = "libero_spatial"
TASK_ID = 0
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256

# 获取 Benchmark 和 Task
benchmark_dict = benchmark.get_benchmark_dict()
task_suite = benchmark_dict[TASK_SUITE_NAME]()
task = task_suite.get_task(TASK_ID)
init_states = task_suite.get_task_init_states(TASK_ID)

print(f"Task Name: {task.name}")
print(f"Task Description: {task.language}")

# 创建环境
env_args = {
    "bddl_file_name": os.path.join(
        os.path.dirname(benchmark.__file__), task.problem_folder, task.bddl_file
    ),
    "render_gpu_device_id": 0,
    "render_mode": None,  # OffScreen rendering
}

env = OffScreenRenderEnv(**env_args)
env.reset()
env.set_init_state(init_states[0])

# 执行一步以确保环境完全初始化
obs, reward, done, info = env.step([0, 0, 0, 0, 0, 0, -1])  # Do nothing action
print("环境初始化完成。")

## 2. 获取 3D 坐标和相机参数并进行投影

我们将获取机器人末端执行器 (End-Effector) 的世界坐标，并将其投影到 `agentview` 相机图像上。

In [ ]:
def project_point_to_image(sim, point_3d, camera_name, width, height):
    """
    将 3D 点投影到相机图像平面。
    """
    # 获取相机投影矩阵 (World -> Pixel)
    # 注意：T_world_to_cam 可能是 4x4 矩阵
    # robosuite 的 camera_utils.get_camera_transform_matrix 返回的是直接到像素坐标的变换矩阵
    P = camera_utils.get_camera_transform_matrix(
        sim=sim, 
        camera_name=camera_name, 
        camera_height=height, 
        camera_width=width
    )
    
    # 构建齐次坐标
    point_homogeneous = np.append(point_3d, 1.0)
    
    # 投影
    point_pixel_homogeneous = P @ point_homogeneous
    
    # 归一化 (除以 w)
    u = point_pixel_homogeneous[0] / point_pixel_homogeneous[2]
    v = point_pixel_homogeneous[1] / point_pixel_homogeneous[2]
    
    return int(u), int(v)

# 获取机器人末端执行器的位置
# 在 Robosuite/Libero 中，通常是 'robot0_eef_pos' 或通过 sim 直接获取
# 这里我们尝试直接从 sim 获取真实的 site 位置，这是最准确的 Ground Truth
robot_eef_site_id = env.sim.model.site_name2id("robot0_eef")
eef_pos_3d = env.sim.data.site_xpos[robot_eef_site_id]

print(f"End-Effector 3D Position (World): {eef_pos_3d}")

# 获取图像
camera_name = "agentview"
# image key 通常是 camera_name + "_image"
image_key = f"{camera_name}_image"

if image_key in obs:
    # 注意：Libero/Robosuite 图像通常是翻转的，需要检查 (H, W, C) 还是其他
    # Robosuite env.step 返回的 obs 中的图像通常是上下颠倒的 (MuJoCo 默认渲染方式)
    # 但 OffScreenRenderEnv 可能会处理这个。我们先直接读取。
    image = obs[image_key]
    
    # 如果图像是浮点数 [0, 1]，转换为 uint8 [0, 255]
    if image.dtype == np.float32 or image.dtype == np.float64:
        image = (image * 255).astype(np.uint8)
    
    # 投影
    u, v = project_point_to_image(env.sim, eef_pos_3d, camera_name, IMAGE_WIDTH, IMAGE_HEIGHT)
    print(f"Projected 2D Pixel: (u={u}, v={v})")
    
    # 可视化
    plt.figure(figsize=(10, 10))
    # Robosuite 渲染出来的图经常是倒着的，我们先按原样显示，看看投影点在哪
    # 如果投影点和图像里的机器人对应不上（比如图像倒了但点没变），说明图像需要翻转
    plt.imshow(image)
    plt.scatter([u], [v], c='r', s=100, marker='x', label='Projected EE Pos')
    plt.title(f"Camera: {camera_name} | EE projection verification")
    plt.legend()
    plt.show()
    
    # 也可以尝试翻转图像看是否对齐
    plt.figure(figsize=(10, 10))
    plt.imshow(np.flipud(image))
    plt.scatter([u], [IMAGE_HEIGHT - v], c='g', s=100, marker='x', label='Projected EE Pos (Flip Check)')
    plt.title(f"Camera: {camera_name} (Flipped) | EE projection verification")
    plt.legend()
    plt.show()

else:
    print(f"Key {image_key} not found in observations. Available keys: {list(obs.keys())}")
